In [ ]:
!pip install -qU openai pinecone-client datasets pinecone-client pyarrow

In [60]:
import openai, pinecone, os
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("API_KEY")

In [ ]:
pinecone.init(
	api_key='90e334e8-7457-4a5c-a336-0d730e3a6606',
	environment='us-west4-gcp-free'
)
index = pinecone.Index('hadid')

In [ ]:
embed_model = "text-embedding-ada-002"

In [ ]:
################
#This loads the embedding and puts it in pinecone

In [ ]:
file_path = 'E:\MyWorks\ReligionBotTry2\Embedding\hadith_correct_order.parquet'
data = pd.read_parquet(file_path)

In [ ]:
data[:3]

In [ ]:
# Create a list of (id, vector, metadata) tuples
list = [{'text':text} for text in data["Text"]]

vectors_to_upsert = [(str(index), row["Embedding"].tolist()) for index, row in data.iterrows()]

# Upsert the vectors
index.upsert(vectors_to_upsert)


In [ ]:
import itertools

# Define the chunking function
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# Your existing data processing
vectors_to_upsert = [(str(index), row["Embedding"].tolist()) for index, row in data.iterrows()]

# Perform the upsert in chunks
for ids_vectors_chunk in chunks(vectors_to_upsert, batch_size=100):
    index.upsert(vectors=ids_vectors_chunk)

In [ ]:
######
#api call w/ context now

In [59]:
text = input("What troubles you?")
res = openai.Embedding.create(input=[text], engine=embed_model)
xq = res['data'][0]['embedding']
ok = index.query(xq, top_k=3,include_values=True, include_metadata=True)

contexts = []
for match in ok['matches']:
    id_num = int(match['id'])
    contexts.append(data.iloc[id_num]["Text"])

context = " ".join(contexts)
completion = openai.ChatCompletion.create(
  model="gpt-4-32k",
  messages=[
    {"role": "system", "content": f"You have been given the context of Islamic Book of hadis. User will ask you questions. Based on the given context, give a straight intelligent 2 sentence answer after understanding the context {context}. Give straight answer"},
    {"role": "user", "content": text}
  ]
)

print(completion.choices[0].message["content"] + '\n' + "Consider these verses in Hadis " + "\n" + contexts[0] + "\n" + contexts[1] + "\n" + contexts[2])

According to the teachings of the Prophet Muhammad (p.b.u.h), everyone finds it easy to do those good deeds that lead them to their destined place for which they have been created. Furthermore, he suggests seeking guidance from Allah in this process by offering a two Rakat prayer and praying to be guided to the actions that are good for you and to be kept away from harm.
Consider these verses in Hadis 
I said, "O Allah's Apostle! Why should a doer (people) try to do good deeds?' The Prophet said, "Everybody will find easy to do such deeds as will lead him to his destined place for which he has been created.'
The Prophet (p.b.u.h) used to teach us the way of doing Istikhara (Istikhara means to ask Allah to guide one to the right sort of action concerning any job or a deed), in all matters as he taught us the Suras of the Quran. He said, "If anyone of you thinks of doing any job he should offer a two Rakat prayer other than the compulsory ones and say (after the prayer): -- 'Allahumma in